# Assignment 2

Github repo for assignment: https://github.com/brentonjackson/csc-4980/tree/master/Assignment2

I'll be using Python for the assignments in this class, as opposed to Matlab.

**Credit goes to Amani Hunter for assets and code**


## Part I:

1. Capture 10s handheld video footage and pan from left to right or right to left.


2. Convert all frames to grayscale. 

    - Pick one frame and find the boundary of any object using Harris corner and Canny edge detection.
   

3. Pick another image frame that has the same object in view as the previous image.
    
   - Find all corresponding points of the object between the two images.
   
   - Find the homography matrix between the images

Below is the code and resulting images from the Harris corner and Canny edge detection functions:

**Recorder.py** records the video footage and allows the user to choose specific frames by pressing SPACE bar

```python
import cv2
import numpy as np
import numba as nb
import depthai as dai

streams = []
streams.append('isp')

@nb.njit(nb.uint16[::1] (nb.uint8[::1], nb.uint16[::1], nb.boolean), parallel=True, cache=True)
def unpack_raw10(input, out, expand16bit):
    lShift = 6 if expand16bit else 0

    for i in np.arange(input.size // 5):
        b4 = input[i * 5 + 4]
        out[i * 4]     = ((input[i * 5]     << 2) | ( b4       & 0x3)) << lShift
        out[i * 4 + 1] = ((input[i * 5 + 1] << 2) | ((b4 >> 2) & 0x3)) << lShift
        out[i * 4 + 2] = ((input[i * 5 + 2] << 2) | ((b4 >> 4) & 0x3)) << lShift
        out[i * 4 + 3] = ((input[i * 5 + 3] << 2) |  (b4 >> 6)       ) << lShift

    return out

print("depthai version:", dai.__version__)
pipeline = dai.Pipeline()

cam = pipeline.createColorCamera()
cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_12_MP)

if 'isp' in streams:
    xout_isp = pipeline.createXLinkOut()
    xout_isp.setStreamName('isp')
    cam.isp.link(xout_isp.input)

device = dai.Device(pipeline)
device.startPipeline()

q_list = []
for s in streams:
    q = device.getOutputQueue(name=s, maxSize=3, blocking=True)
    q_list.append(q)
    cv2.namedWindow(s, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(s, (960, 540))

capture_flag = False
image_counter = 0
while True:
    for q in q_list:
        name = q.getName()
        data = q.get()
        width, height = data.getWidth(), data.getHeight()
        payload = data.getData()
        capture_file_info_name = f"Amani_capture_{name}_{image_counter}"
        if name == 'isp':
            shape = (height * 3 // 2, width)
            yuv420p = payload.reshape(shape).astype(np.uint8)
            bgr = cv2.cvtColor(yuv420p, cv2.COLOR_YUV2BGR_IYUV)
            grayscale_img = cv2.cvtColor(bgr,cv2.COLOR_BGR2GRAY)
        if capture_flag:
            filename = capture_file_info_name + '.png'
            grayscale_img = np.ascontiguousarray(grayscale_img)
            cv2.imwrite(filename, grayscale_img)
        bgr = np.ascontiguousarray(bgr)
        cv2.imshow(name, grayscale_img)
    capture_flag = False
    key = cv2.waitKey(5)
    if key == ord('q'):
        break
    elif key%256 == 32:
        capture_flag = True
        image_counter += 1
  
```

**HarrisCorner.py**:

```python
import numpy as np
import cv2
import matplotlib.pyplot as plt

captured_image = cv2.imread("Amani_capture_isp_2.png")
grayImg = cv2.cvtColor(captured_image, cv2.COLOR_BGR2GRAY)
grayImg = np.float32(grayImg)
dest = cv2.cornerHarris(grayImg, 8, 29, 0.05)
dest = cv2.dilate(dest, None)
captured_image[dest > 0.01 * dest.max()] = [0, 0, 255]
cv2.imshow('HarrisCorners', captured_image)
cv2.imwrite("HarrisCorners.png", captured_image)

plt.imshow(captured_image)
plt.show()
```

**CannyEdgeDetection.py**:

```python
import cv2
import matplotlib.pyplot as plt

captured_image = cv2.imread("Amani_capture_isp_1.png",0)
edges = cv2.Canny(captured_image, 180, 200)
plt.subplot(121),plt.imshow(captured_image,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
plt.show()
```

Input image:

![Input image](Part-1/Amani_capture_isp_2.png)

Output image:

![Output image](Part-1/HarrisCorners.png)

**Coordinates.py** allows the user to click on the 4 coordinates of the object in both of the images.
The coordinates will be used to find the homography matrix.

```python
import cv2

def clickImage(event, x, y, flags, params):
	if event == cv2.EVENT_LBUTTONDOWN:
		print(x, ' ', y)
		font = cv2.FONT_HERSHEY_SIMPLEX
		cv2.putText(img, str(x) + ',' +
					str(y), (x,y), font,
					1, (255, 0, 0), 2)
		cv2.imshow('image', img)
	if event==cv2.EVENT_RBUTTONDOWN:
		print(x, ' ', y)
		font = cv2.FONT_HERSHEY_SIMPLEX
		b = img[y, x, 0]
		g = img[y, x, 1]
		r = img[y, x, 2]
		cv2.putText(img, str(b) + ',' +
					str(g) + ',' + str(r),
					(x,y), font, 1,
					(255, 255, 0), 2)
		cv2.imshow('image', img)

#img = cv2.imread('Amani_capture_isp_1.png', 1)
img = cv2.imread('Amani_capture_isp_2.png', 1)

cv2.imshow('image', img)
cv2.setMouseCallback('image', clickImage)
cv2.waitKey(0)
cv2.destroyAllWindows()
```


**HomographyMatrix.py**
```python
import cv2
import numpy as np
import matplotlib.pyplot as plt

src_points = np.array([[1335,978],[1333,1928],[2989,1950],[2988,952]])
dest_points = np.array([[299,856],[329,1948],[2131,1878],[2122,896]])

h, status = cv2.findHomography(src_points, dest_points)

im_src = cv2.imread('Amani_capture_isp_1.png')
im_dst = cv2.imread('Amani_capture_isp_2.png')

im_out = cv2.warpPerspective(im_src, h, (im_dst.shape[1],im_dst.shape[0]))
print(h)
cv2.imshow("Warped_Source_Image", im_out)
plt.imshow(im_out)
plt.show()

#HM
#[[ 1.55385410e+00  4.56897321e-02 -1.76721065e+03]
#[ 1.58042282e-01  1.38417516e+00 -5.57351990e+02]
#[ 1.20560432e-04  1.62286723e-05  1.00000000e+00]]

```

## Part II:

Implement the image stitching application in MATLAB (not necessary to be real-time).

Test your application for any FIVE of a set of 3 image-set available in the gsu_building_database.

**Stitching.py:**
```python
import numpy as np
import cv2

imagesPaths1 = ["./GSU_Buildings/Set2/rialto1.jpeg","./GSU_Buildings/Set2/rialto2.jpeg","./GSU_Buildings/Set2/rialto3.jpeg"]
imagesPaths2 = ["./GSU_Buildings/Set1/studentcentereast1.jpg","./GSU_Buildings/Set1/studentcentereast2.jpg","./GSU_Buildings/Set1/studentcentereast3.jpg"]
imagesPaths3 = ["./GSU_Buildings/Set3/bookstore1.jpg","./GSU_Buildings/Set3/bookstore2.jpg","./GSU_Buildings/Set3/bookstore3.jpg"]
imagesPaths4 = ["./GSU_Buildings/Set4/bookstore5.jpg","./GSU_Buildings/Set4/bookstore6.jpg","./GSU_Buildings/Set4/bookstore7.jpg"]
imagesPaths5 = ["./GSU_Buildings/Set5/tdeck1.jpg","./GSU_Buildings/Set5/tdeck2.jpg","./GSU_Buildings/Set5/tdeck3.jpg"]
images = []

for path in imagesPaths2:
	image = cv2.imread(path)
	images.append(image)

stitcher = cv2.Stitcher_create()
(status, stitched) = stitcher.stitch(images)
print(stitched)
if status == 0:
	cv2.imshow("Stitched images", stitched)
	cv2.waitKey(0)
else:
	print("Failed to strich")

```

Images before stitching:

![image1](Part-2/GSU_Buildings/Set1/studentcentereast1.jpg)
![image2](Part-2/GSU_Buildings/Set1/studentcentereast2.jpg)
![image3](Part-2/GSU_Buildings/Set1/studentcentereast3.jpg)

Output of stitched images:

![Output image](Part-2/stitched.png)

## Part III:

Implement an application that will compute and display the INTEGRAL image feed along with the stereo and RGB feed.

**IntegralImageFeed.py**
```python
import cv2
import numpy as np
import numba as nb
import depthai as dai

def integral_image(image, *, dtype=None):
    if dtype is None and image.real.dtype.kind == 'f':
        dtype = np.promote_types(image.dtype, np.float64)
    S = image
    for i in range(image.ndim):
        S = S.cumsum(axis=i, dtype=dtype)
    return S


def integrate(ii, start, end):
    start = np.atleast_2d(np.array(start))
    end = np.atleast_2d(np.array(end))
    rows = start.shape[0]
    total_shape = ii.shape
    total_shape = np.tile(total_shape, [rows, 1])
    start_negatives = start < 0
    end_negatives = end < 0
    start = (start + total_shape) * start_negatives + \
             start * ~(start_negatives)
    end = (end + total_shape) * end_negatives + \
           end * ~(end_negatives)

    if np.any((end - start) < 0):
        print('Error')
    S = np.zeros(rows)
    bit_perm = 2 ** ii.ndim
    width = len(bin(bit_perm - 1)[2:])
    for i in range(bit_perm):
        binary = bin(i)[2:].zfill(width)
        bool_mask = [bit == '1' for bit in binary]
        sign = (-1)**sum(bool_mask)
        bad = [np.any(((start[r] - 1) * bool_mask) < 0)
               for r in range(rows)]
        corner_points = (end * (np.invert(bool_mask))) + \
                         ((start - 1) * bool_mask)
        S += [sign * ii[tuple(corner_points[r])] if(not bad[r]) else 0
              for r in range(rows)]
    return S

streams = []
streams.append('isp')
@nb.njit(nb.uint16[::1] (nb.uint8[::1], nb.uint16[::1], nb.boolean), parallel=True, cache=True)
def unpack_raw10(input, out, expand16bit):
    lShift = 6 if expand16bit else 0
    for i in np.arange(input.size // 5):
        b4 = input[i * 5 + 4]
        out[i * 4] = ((input[i * 5] << 2) | (b4 & 0x3)) << lShift
        out[i * 4 + 1] = ((input[i * 5 + 1] << 2) | ((b4 >> 2) & 0x3)) << lShift
        out[i * 4 + 2] = ((input[i * 5 + 2] << 2) | ((b4 >> 4) & 0x3)) << lShift
        out[i * 4 + 3] = ((input[i * 5 + 3] << 2) | (b4 >> 6)) << lShift

    return out

pipeline = dai.Pipeline()
cam = pipeline.createColorCamera()
cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_12_MP)

if 'isp' in streams:
    xout_isp = pipeline.createXLinkOut()
    xout_isp.setStreamName('isp')
    cam.isp.link(xout_isp.input)

device = dai.Device(pipeline)
device.startPipeline()

q_list = []
for s in streams:
    q = device.getOutputQueue(name=s, maxSize=3, blocking=True)
    q_list.append(q)
    cv2.namedWindow(s, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(s, (960, 540))

capture_flag = False
img_counter = 0
while True:
    for q in q_list:
        name = q.getName()
        data = q.get()
        width, height = data.getWidth(), data.getHeight()
        payload = data.getData()
        capture_file_info_str = f"Amani_capture_{name}_{img_counter}"
        if name == 'isp':
            shape = (height * 3 // 2, width)
            yuv420p = payload.reshape(shape).astype(np.uint8)
            bgr = cv2.cvtColor(yuv420p, cv2.COLOR_YUV2BGR_IYUV)
            frame = cv2.copyMakeBorder(bgr, 50, 50, 50, 50, cv2.BORDER_CONSTANT, (0,0,0))
            frame = integral_image(frame)
            frame = frame/np.amax(frame)
            frame = np.clip(frame, 0,255)
        if capture_flag:
            filename = capture_file_info_str + '.png'
            grayscale_img = np.ascontiguousarray(grayscale_img)
            cv2.imwrite(filename, grayscale_img)
        bgr = np.ascontiguousarray(bgr)
        cv2.imshow(name, frame)
    capture_flag = False
    key = cv2.waitKey(5)
    if key % 256 == 27:
        break
    elif key%256 == 32:
        capture_flag = True
        img_counter += 1
```

## Part IV:

Implement the image stitching, for at least 1 pair of images. Use SIFT features.

Images before stitching:

![Input image 1](Part-1/Amani_capture_isp_1.png)
![Input image 2](Part-1/Amani_capture_isp_2.png)

Image after stitching:


![Output image](Part-4/output.png)


## Part V:

Repeat (4) using ORB features.

**OrbImageStitching.py**
```python
import imutils
import cv2
import numpy as np
import matplotlib.pyplot as plt

cv2.ocl.setUseOpenCL(False)

def getHomography(kps_train, kps_query, matches, reprojThresh):
    kpsA = np.float32([kp.pt for kp in kps_train])
    kpsB = np.float32([kp.pt for kp in kps_query])
    if len(matches) > 4:
        pointsA = np.float32([kpsA[m.queryIdx] for m in matches])
        pointsB = np.float32([kpsB[m.trainIdx] for m in matches])
        (Homography, status) = cv2.findHomography(pointsA, pointsB, cv2.RANSAC, reprojThresh)
        print('matches: ', matches)
        return (matches, Homography, status)
    else:
        return None

def GetImageKeyPoints(image):
    descriptor = cv2.ORB_create()
    kps, features = descriptor.detectAndCompute(image, None)
    return (kps, features)

def MatchImageKeyPoints(features_train, features_query, ratio):
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    raw_match = bf.knnMatch(features_train, features_query, 2)
    matches = []

    for m, n in raw_match:
        if m.distance < n.distance * ratio:
            matches.append(m)

    return matches

def TransformImageToGrayScale(result):
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]
    cnts = cv2.findContours(threshold.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    (x, y, w, h) = cv2.boundingRect(c)
    result = result[y:y + h, x:x + w]
    return result


img_train = cv2.imread("../Part-1/Amani_capture_isp_1.png")
img_query = cv2.imread("../Part-1/Amani_capture_isp_2.png")
gray_img_train = cv2.cvtColor(img_train, cv2.COLOR_RGB2GRAY)
gray_img_query = cv2.cvtColor(img_query, cv2.COLOR_RGB2GRAY)
kps_train, features_train = GetImageKeyPoints(gray_img_train)
kps_query, features_query = GetImageKeyPoints(gray_img_query)
print(features_train,features_query, kps_train, kps_query)
matches = MatchImageKeyPoints(features_train, features_query, 0.75)
print(matches)
temp_img = cv2.drawMatches(img_train, kps_train, img_query, kps_query, np.random.choice(matches, 100), None,
                           flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
M = getHomography(kps_train, kps_query, matches, 4)
matches, homography, status = M
width = img_train.shape[1] + img_query.shape[1]
height = img_train.shape[0] + img_query.shape[0]
imageResult = cv2.warpPerspective(img_train, homography, (width, height))
imageResult[0:img_query.shape[0], 0:img_query.shape[1]] = img_query
imageResult = TransformImageToGrayScale(imageResult)
plt.imshow(imageResult)
plt.show()
cv2.imwrite("orboutput.png", imageResult)

```

Image after stitching (same input images as above):


![Output image](Part-5/orboutput.png)
